In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/IRI/output/"
FILES_PATH = "/content/drive/MyDrive/Colab Notebooks/IRI/"

In [ ]:
! pip3 install textattack

In [ ]:
! pip install sentence_transformers

In [ ]:
! pip install transformers[torch]

In [6]:
! gdown "165kzfZDsRTZAAfZKedeZiUlKzMcHNgPd"  # Arabic_stop_words.txt
! gdown "1MPHZcco5Rh8VGye91qc0bxXd0iIECD3-"  # correct_classified_test.csv
! gdown "10Umn1MBzzMOnb0l-VtcrSfMk41LhhnhF"  # arabic_offensive_lang_detection_arabert.pt (bert-base-arabertv02-twitter model)

Downloading...
From: https://drive.google.com/uc?id=165kzfZDsRTZAAfZKedeZiUlKzMcHNgPd
To: /content/Arabic_stop_words.txt
100% 6.48k/6.48k [00:00<00:00, 31.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MPHZcco5Rh8VGye91qc0bxXd0iIECD3-
To: /content/correct_classified_test.csv
100% 243k/243k [00:00<00:00, 94.5MB/s]
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=10Umn1MBzzMOnb0l-VtcrSfMk41LhhnhF 



In [7]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoConfig, BertForSequenceClassification,AutoModelForMaskedLM


import textattack
from textattack import Attack
from textattack import datasets
from textattack import Attacker, AttackArgs
from textattack.attack_recipes import A2TYoo2021
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import AttackRecipe
from textattack.models.wrappers import ModelWrapper
from textattack.transformations import WordSwapWordNet
from textattack.search_methods import GreedyWordSwapWIR
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.goal_functions import UntargetedClassification
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import BERT
from textattack.transformations import WordSwapEmbedding  #, WordSwapMaskedLM
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    MaxModificationRate,
    RepeatModification,
    StopwordModification,
)
from textattack.shared import utils
import itertools
# from itertools import zip_longest
from textattack.transformations.word_swaps import WordSwap


pd.set_option('display.max_colwidth', None)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
arabic_stop_words=[]
with open ('./Arabic_stop_words.txt',encoding='utf-8') as f :
    for word in f.readlines() :
        arabic_stop_words.append(word.split("\n")[0])

In [9]:
PREFIX_LIST = [
    "ال",
    "و",
    "ف",
    "ب",
    "ك",
    "ل",
    "لل",
    "\u0627\u0644",
    "\u0648",
    "\u0641",
    "\u0628",
    "\u0643",
    "\u0644",
    "\u0644\u0644",
    "س",
]
SUFFIX_LIST = [
    "ه",
    "ها",
    "ك",
    "ي",
    "هما",
    "كما",
    "نا",
    "كم",
    "هم",
    "هن",
    "كن",
    "ا",
    "ان",
    "ين",
    "ون",
    "وا",
    "ات",
    "ت",
    "ن",
    "ة",
    "\u0647",
    "\u0647\u0627",
    "\u0643",
    "\u064a",
    "\u0647\u0645\u0627",
    "\u0643\u0645\u0627",
    "\u0646\u0627",
    "\u0643\u0645",
    "\u0647\u0645",
    "\u0647\u0646",
    "\u0643\u0646",
    "\u0627",
    "\u0627\u0646",
    "\u064a\u0646",
    "\u0648\u0646",
    "\u0648\u0627",
    "\u0627\u062a",
    "\u062a",
    "\u0646",
    "\u0629",
]


# the never_split list is used with the transformers library
_PREFIX_SYMBOLS = [x + "+" for x in PREFIX_LIST]
_SUFFIX_SYMBOLS = ["+" + x for x in SUFFIX_LIST]
NEVER_SPLIT_TOKENS = list(set(_PREFIX_SYMBOLS + _SUFFIX_SYMBOLS))

In [14]:
model_name = "aubmindlab/bert-base-arabertv02-twitter"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          do_lower_case=False,
                                          do_basic_tokenize=True,
                                          never_split=NEVER_SPLIT_TOKENS)

In [ ]:
model = torch.load(FILES_PATH+'arabic_offensive_lang_detection_arabert.pt',map_location=torch.device('cpu') )
model.to('cpu')

In [16]:
def data_reformat(data):
    # create a list of data tubles
    data_tuples = [(text, label) for text, label in zip(data["tweet_clean"], data["class"])]
    # return a dataset
    return datasets.Dataset(data_tuples)

In [ ]:
# load test data
correct_classified_test = pd.read_csv('correct_classified_test.csv')
correct_classified_test = correct_classified_test.reset_index(drop=True)

attack_dataset = data_reformat(correct_classified_test)

# for example in attack_dataset:
#     print("label: ", example[1], " -- text: ",example[0]['text'])

In [48]:
class WordSwapMaskedLM_Modified(WordSwap):
    def __init__(
        self,
        method,
        masked_language_model,
        tokenizer=None,
        max_length=512,
        window_size=float("inf"),
        max_candidates=20,
        # min_confidence=5e-4,
        batch_size=16,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.method = method
        self.max_length = max_length
        self.window_size = window_size
        self.max_candidates = max_candidates
        # self.min_confidence = min_confidence
        self.batch_size = batch_size

        if isinstance(masked_language_model, str):
            self._language_model = AutoModelForMaskedLM.from_pretrained(
                masked_language_model
            )
            self._lm_tokenizer = AutoTokenizer.from_pretrained(
                masked_language_model, use_fast=True
            )
        else:
            self._language_model = masked_language_model
            if tokenizer is None:
                raise ValueError(
                    "`tokenizer` argument must be provided when passing an actual model as `masked_language_model`."
                )
            self._lm_tokenizer = tokenizer
        self._language_model.to(utils.device)
        self._language_model.eval()
        self.masked_lm_name = self._language_model.__class__.__name__

    def _encode_text(self, text):
        encoding = self._lm_tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        return encoding.to(utils.device)



    def _bae_replacement_words(self, current_text, indices_to_modify):
        masked_texts = []
        for index in indices_to_modify:
            masked_text = current_text.replace_word_at_index(index, self._lm_tokenizer.mask_token)
            masked_texts.append(masked_text.text)
        i = 0
        # 2-D list where for each index to modify we have a list of replacement words
        replacement_words = []
        while i < len(masked_texts):
            inputs = self._encode_text(masked_texts[i : i + self.batch_size])
            ids = inputs["input_ids"].tolist()
            with torch.no_grad():
                preds = self._language_model(**inputs)[0]

            for j in range(len(ids)):
                try:
                    # Need try-except b/c mask-token located past max_length might be truncated by tokenizer
                    masked_index = ids[j].index(self._lm_tokenizer.mask_token_id)
                except ValueError:
                    replacement_words.append([])
                    continue

                mask_token_logits = preds[j, masked_index]
                mask_token_probs = torch.softmax(mask_token_logits, dim=0)
                ranked_indices = torch.argsort(mask_token_probs, descending=True)
                top_words = []
                for _id in ranked_indices:
                    _id = _id.item()
                    word = self._lm_tokenizer.convert_ids_to_tokens(_id)
                    if utils.check_if_subword(word,self._language_model.config.model_type,(masked_index == 1)):
                        word = utils.strip_BPE_artifacts(word, self._language_model.config.model_type)
                    # if (mask_token_probs[_id] >= self.min_confidence and utils.is_one_word(word)and not utils.check_if_punctuations(word)):
                    if (utils.is_one_word(word)and not utils.check_if_punctuations(word)):
                        top_words.append(word)

                    # if (len(top_words) >= self.max_candidates or mask_token_probs[_id] < self.min_confidence):
                    if (len(top_words) >= self.max_candidates):
                        break
                replacement_words.append(top_words)

            i += self.batch_size

        return replacement_words



    def _bert_attack_replacement_words(self, current_text, index, id_preds, masked_lm_logits):
        """Get replacement words for the word we want to replace using BERT-
        Attack method.

        Args:
            current_text (AttackedText): Text we want to get replacements for.
            index (int): index of word we want to replace
            id_preds (torch.Tensor): N x K tensor of top-K ids for each token-position predicted by the masked language model.
                N is equivalent to `self.max_length`.
            masked_lm_logits (torch.Tensor): N x V tensor of the raw logits outputted by the masked language model.
                N is equivlaent to `self.max_length` and V is dictionary size of masked language model.
        """
        # We need to find which BPE tokens belong to the word we want to replace
        masked_text = current_text.replace_word_at_index(index, self._lm_tokenizer.mask_token)
        current_inputs = self._encode_text(masked_text.text)
        current_ids = current_inputs["input_ids"].tolist()[0]
        word_tokens = self._lm_tokenizer.encode(current_text.words[index], add_special_tokens=False)

        try:
            # Need try-except b/c mask-token located past max_length might be truncated by tokenizer
            masked_index = current_ids.index(self._lm_tokenizer.mask_token_id)
        except ValueError:
            return []

        # List of indices of tokens that are part of the target word
        target_ids_pos = list(range(masked_index, min(masked_index + len(word_tokens), self.max_length)))

        if not len(target_ids_pos):
            return []
        elif len(target_ids_pos) == 1:
            # Word to replace is tokenized as a single word
            top_preds = id_preds[target_ids_pos[0]].tolist()
            replacement_words = []
            for id in top_preds:
                token = self._lm_tokenizer.convert_ids_to_tokens(id)
                if utils.is_one_word(token) and not utils.check_if_subword(token, self._language_model.config.model_type, index == 0):
                    replacement_words.append(token)
            return replacement_words
        else:
            # Word to replace is tokenized as multiple sub-words
            top_preds = [id_preds[i] for i in target_ids_pos]
            products = itertools.product(*top_preds)
            combination_results = []
            # Original BERT-Attack implement uses cross-entropy loss
            cross_entropy_loss = torch.nn.CrossEntropyLoss(reduction="none")
            target_ids_pos_tensor = torch.tensor(target_ids_pos)
            word_tensor = torch.zeros(len(target_ids_pos), dtype=torch.long)
            for bpe_tokens in products:
                for i in range(len(bpe_tokens)):
                    word_tensor[i] = bpe_tokens[i]

                logits = torch.index_select(masked_lm_logits, 0, target_ids_pos_tensor)
                loss = cross_entropy_loss(logits, word_tensor)
                perplexity = torch.exp(torch.mean(loss, dim=0)).item()
                word = "".join(self._lm_tokenizer.convert_ids_to_tokens(word_tensor)).replace("##", "")
                if utils.is_one_word(word):
                    combination_results.append((word, perplexity))
            # Sort to get top-K results
            sorted(combination_results, key=lambda x: x[1])
            top_replacements = [x[0] for x in combination_results[: self.max_candidates]]
            return top_replacements

    def _get_transformations(self, current_text, indices_to_modify):
        indices_to_modify = list(indices_to_modify)

        if self.method == "bert-attack":
            current_inputs = self._encode_text(current_text.text)
            with torch.no_grad():
                pred_probs = self._language_model(**current_inputs)[0][0]
            top_probs, top_ids = torch.topk(pred_probs, self.max_candidates)
            id_preds = top_ids.cpu()
            masked_lm_logits = pred_probs.cpu()

            transformed_texts = []

            for i in indices_to_modify:
                word_at_index = current_text.words[i]
                replacement_words = self._bert_attack_replacement_words(current_text, i, id_preds=id_preds, masked_lm_logits=masked_lm_logits,)

                for r in replacement_words:
                    r = r.strip("Ġ")
                    if r != word_at_index:
                        transformed_texts.append(current_text.replace_word_at_index(i, r))

            return transformed_texts

        elif self.method == "bae":
            replacement_words = self._bae_replacement_words(current_text, indices_to_modify)
            transformed_texts = []
            #print(f'top words [replacement words]: {replacement_words}')
            for i in range(len(replacement_words)):
                index_to_modify = indices_to_modify[i]
                word_at_index = current_text.words[index_to_modify]
                for word in replacement_words[i]:
                    word = word.strip("Ġ")
                    if (
                        word != word_at_index
                        #and re.search("[a-zA-Z]", word)
                        and word != '[UNK]'
                        and len(utils.words_from_text(word)) == 1
                    ):
                        transformed_texts.append(current_text.replace_word_at_index(index_to_modify, word))
            return transformed_texts
        else:
            raise ValueError(f"Unrecognized value {self.method} for `self.method`.")

    def extra_repr_keys(self):
        return [
            "method",
            "masked_lm_name",
            "max_length",
            "max_candidates",
            # "min_confidence",
        ]


    def recover_word_case(word, reference_word):
        """Makes the case of `word` like the case of `reference_word`.

        Supports lowercase, UPPERCASE, and Capitalized.
        """
        if reference_word.islower():
            return word.lower()
        elif reference_word.isupper() and len(reference_word) > 1:
            return word.upper()
        elif reference_word[0].isupper() and reference_word[1:].islower():
            return word.capitalize()
        else:
            # if other, just do not alter the word's case
            return word

In [49]:
"""
A2T (A2T: Attack for Adversarial Training Recipe)
==================================================
"""
class A2TYoo2021(AttackRecipe):

    @staticmethod
    def build(model_wrapper, model_name, method, threshold = 0.5):
        """Build attack recipe.

        Args:
            model_wrapper (:class:`~textattack.models.wrappers.ModelWrapper`):
                Model wrapper containing both the model and the tokenizer.
            mlm (:obj:`bool`, `optional`, defaults to :obj:`False`):
                If :obj:`True`, load `A2T-MLM` attack. Otherwise, load regular `A2T` attack.
        Returns:
            :class:`~textattack.Attack`: A2T attack.
        """
        constraints = [RepeatModification(), StopwordModification(stopwords = arabic_stop_words)]
        constraints.append(PartOfSpeech(allow_verb_noun_swap=False))
        constraints.append(MaxModificationRate(max_rate=0.1, min_threshold=4))
        sent_encoder = BERT(model_name='symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli', threshold=threshold, metric="cosine")
        constraints.append(sent_encoder)

        transformation = WordSwapMaskedLM_Modified(method=method, masked_language_model=model_name)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper, model_batch_size=32)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="gradient")

        return Attack(goal_function, constraints, transformation, search_method)

In [50]:
model_name

'aubmindlab/bert-base-arabertv02-twitter'

In [51]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

In [52]:
def start_attack(model_name, method, output_file, threshold = 0.5):
  attack = A2TYoo2021.build(model_wrapper, model_name, method, threshold)
  attack.transformation.language = 'arb'

  attack_args = textattack.AttackArgs(
    num_examples=710,
    log_to_csv= SAVE_PATH + output_file,
    disable_stdout=True
  )

  attacker = Attacker(attack, attack_dataset, attack_args)
  attacker.attack_dataset()

#### bae method:

In [53]:
"""
model_name = "aubmindlab/bert-base-arabertv02-twitter",
method = "bae"
threshold = 0.5
output_file = "bae_results.csv"
"""

start_attack(model_name, 'bae', "bae_results.csv")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/Colab Notebooks/IRI/output/bae_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  gradient
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM_Modified(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  20
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  False
        (compare_against_original):  True
      )
    (1): BERT(
        (metric):  cosine
        (threshold):  0.5
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
    (4): MaxModificationRate(
        (max_rate):  0.1
        (min_threshold):  4
      )
  (is_black_box):  False
) 



  0%|          | 0/710 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
[Succeeded / Failed / Skipped / Total] 539 / 162 / 9 / 710: 100%|██████████| 710/710 [26:00<00:00,  2.20s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 539    |
| Number of failed attacks:     | 162    |
| Number of skipped attacks:    | 9      |
| Original accuracy:            | 98.73% |
| Accuracy under attack:        | 22.82% |
| Attack success rate:          | 76.89% |
| Average perturbed word %:     | 18.71% |
| Average num. words per input: | 14.95  |
| Avg num queries:              | 41.68  |
+-------------------------------+--------+


In [54]:
bae_results = pd.read_csv(SAVE_PATH+"bae_results.csv")

In [55]:
bae_results["result_type"].value_counts()

Successful    539
Failed        162
Skipped         9
Name: result_type, dtype: int64

In [56]:
bae_results[bae_results['result_type'] == 'Successful'].head(10)

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,وش ذا والله لو مو شايفات رجال حلوين بحياتهم استغفر الله قله [[حياء]] [[وتخلف]] 👎🏻👎🏻👎🏻😡😡 بايع الكليجا,وش ذا والله لو مو شايفات رجال حلوين بحياتهم استغفر الله قله [[بسم]] [[الله]] 👎🏻👎🏻👎🏻😡😡 بايع الكليجا,0.001713,0.681959,1,0,1,41,Successful
2,بايع الكليجا ايه بعدين تعالوا قولوا انتم [[شهوانين]] 😡,بايع الكليجا ايه بعدين تعالوا قولوا انتم [[بشر]] 😡,0.016940,0.983896,1,0,1,17,Successful
3,والله عيب اللي صار بغض النظر جميل او شين [[وين]] كرامتك وحياءك يعني بعض البنات اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل [[تقرفت]] من بعض [[الهمج]] اللي [[بالفيديو]] بايع الكليجا الجنادريه,والله عيب اللي صار بغض النظر جميل او شين [[يعني]] كرامتك وحياءك يعني بعض البنات اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل [[مو]] من بعض [[الشباب]] اللي [[مو]] بايع الكليجا الجنادريه,0.002574,0.519205,1,0,1,79,Successful
6,ويعه 😤👊🏻👊🏻👊🏻 ياربي شقد [[اكررهك]] 😖😖,ويعه 😤👊🏻👊🏻👊🏻 ياربي شقد [[كذا]] 😖😖,0.358952,0.998139,1,0,1,17,Successful
8,[[موت]] موت وع [[يتميلح]] بشكل [[مقرفف]] 🤢,[[ه]] موت وع [[ع]] بشكل [[كبير]] 🤢,0.003080,0.977395,1,0,1,54,Successful
9,عندما تغرد [[الحمير]] يكذب الكذبه و يصدقها 🐕🐕,عندما تغرد [[انسان]] يكذب الكذبه و يصدقها 🐕🐕,0.003553,0.976087,1,0,1,21,Successful
11,[[ساقطه]] عيب عليك تقذفها [[يا]] كليب 🐕,[[ما]] عيب عليك تقذفها [[من]] كليب 🐕,0.001467,0.562857,1,0,1,73,Successful
12,بايع الكليجا ايوالله كلكم [[تفله]] عليكم ياللي حاقين ع [[ام]] غمزه وع [[ايفانكا]] وبياع الكليجا كلكم [[زبايل]] 😷,بايع الكليجا ايوالله كلكم [[مبروك]] عليكم ياللي حاقين ع [[الدنيا]] غمزه وع [[تويتر]] وبياع الكليجا كلكم [[بعد]] 😷,0.001003,0.561300,1,0,1,80,Successful
13,سالو واحد ما هو الكاءن الذي يزن كيلو في الصباح و غرام في الليل فاجاب علي السءال المراه زوجها يقول لهالصبح قومي يا [[بقره]] 🐄 وفي الليل تعالي يا كتكوته 😂😂💔,سالو واحد ما هو الكاءن الذي يزن كيلو في الصباح و غرام في الليل فاجاب علي السءال المراه زوجها يقول لهالصبح قومي يا [[ديك]] 🐄 وفي الليل تعالي يا كتكوته 😂😂💔,0.196582,0.771324,1,0,1,21,Successful
14,عاوزين يقللوا شعبيته عشان الانتخابات الرءاسيه يقوموا يقولوا هيفرج عن مرسي وجماعته [[كفايه]] [[هبل]] [[وعبط]] [[زهقنا]] 😡,عاوزين يقللوا شعبيته عشان الانتخابات الرءاسيه يقوموا يقولوا هيفرج عن مرسي وجماعته [[بدل]] [[تأليف]] [[الكلام]] [[عليه]] 😡,0.005379,0.503591,1,0,1,81,Successful


In [57]:
bae_results[bae_results['result_type'] == 'Failed'].head(10)

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
1,احلي [[شعور]] لما [[اوسع]] خرقك [[الضيق]] بزبي الكبير ياخنيث 😈👠👠 [[عمه]] كسي ديوث دياثه ممحونه كلبه كلب عمتك ساديه سالب,احلي [[شئ]] لما [[افر]] خرقك [[وادخله]] بزبي الكبير ياخنيث 😈👠👠 [[[رابط]]] كسي ديوث دياثه ممحونه كلبه كلب عمتك ساديه سالب,0.001970,0.002577,1,1,1,100,Failed
4,هءلاء هم مايسمي علماء الوهابيه داءمانظرتهم للمراه دونيه [[وبهيمه]] كيف ترتقي الامه العربيه بهءلاء الشرذمه اللذين يسمون انفسهم ب علماء امحق علم هءلاء سفهاء ومن يصدقهم يكون [[بهيمه]] ✋🏻 اقترب [[الظهور]] العلني ل المهدي وسيحررنامن هذه [[العقول]] المعفنه 😷,هءلاء هم مايسمي علماء الوهابيه داءمانظرتهم للمراه دونيه [[ف]] كيف ترتقي الامه العربيه بهءلاء الشرذمه اللذين يسمون انفسهم ب علماء امحق علم هءلاء سفهاء ومن يصدقهم يكون [[مخطئ]] ✋🏻 اقترب [[الموعد]] العلني ل المهدي وسيحررنامن هذه [[المعتقدات]] المعفنه 😷,0.001202,0.002183,1,1,1,79,Failed
5,[[ازق]] من كذا [[تحريض]] مافيه [[الله]] [[يمرض]] اشكالك 😷,[[الطف]] من كذا [[ضحك]] مافيه [[شي]] [[يعين]] اشكالك 😷,0.001451,0.004723,1,1,1,39,Failed
7,بايع الكليجا وصلنا لزمن نقول الله [[لايبلانا]] اعرف الحرمه من زود الحياء تستحي تحط عينها بعين اي رجال مو محرم لها [[وهذولا]] تصوير وقله حياء وتربيه [[فضحونا]] [[مكبوتات]] 😷,بايع الكليجا وصلنا لزمن نقول الله [[ستر]] اعرف الحرمه من زود الحياء تستحي تحط عينها بعين اي رجال مو محرم لها [[لكن]] تصوير وقله حياء وتربيه [[عيب]] [[كثير]] 😷,0.001376,0.061224,1,1,1,80,Failed
10,والله ابركها من ساعه من [[لمن]] تفكريه تخليه يالمغثه كيفه [[خويه]] يابزر مالك صلاح الله ياخذ ذا [[الاشكال]] [[قررف]] متء تموتون 😷😷,والله ابركها من ساعه من [[الصبح]] تفكريه تخليه يالمغثه كيفه [[انا]] يابزر مالك صلاح الله ياخذ ذا [[المرض]] [[صدق]] متء تموتون 😷😷,0.001202,0.002781,1,1,1,80,Failed
18,انتم نجستم اليمن اي سيده تتكلم عنها يا [[معتوه]] [[زيد]] هل بنظرك علي اساس سيدك يطلع ملك قبحت انت وسيدك يا [[كلب]] اليمن العنه عليك 🖕🖕 اجلس [[فوقها]],انتم نجستم اليمن اي سيده تتكلم عنها يا [[صالح]] [[الحوثي]] هل بنظرك علي اساس سيدك يطلع ملك قبحت انت وسيدك يا [[فرعون]] اليمن العنه عليك 🖕🖕 اجلس [[بصنعاء]],0.001175,0.001814,1,1,1,81,Failed
20,برنامج يسعد صباحك عمره ماسستفزني متصصل مثل سلطان اش هالاسسلوب المستفز والمنحطط [[يالطيف]] الله يعين العالم الي معاك تتعامل مو بس [[مرتك]] هذي لها الجنه بحول الله [[اعوذبالله]] من [[هالاشكال]] 😷,برنامج يسعد صباحك عمره ماسستفزني متصصل مثل سلطان اش هالاسسلوب المستفز والمنحطط [[فيه]] الله يعين العالم الي معاك تتعامل مو بس [[لك]] هذي لها الجنه بحول الله [[لاتخاف]] من [[الحسد]] 😷,0.001489,0.005804,1,1,1,78,Failed
24,المملكه العربيه السعوديه مهما نابحت يا [[الكلب]] المرتزق الماجور [[تضل]] هي البلد الوحيد الي خيره وصل القاصي والداني يا ناكر الجميل يا [[بوق]] [[الغرب]] المقيت 💩👺,المملكه العربيه السعوديه مهما نابحت يا [[الصهيوني]] المرتزق الماجور [[فأنت]] هي البلد الوحيد الي خيره وصل القاصي والداني يا ناكر الجميل يا [[عدو]] [[التطبيع]] المقيت 💩👺,0.001544,0.002224,1,1,1,79,Failed
26,[[وش]] السخافه [[والقرفخفيفات]] بشكل 😷,[[من]] السخافه [[ه]] بشكل 😷,0.001876,0.147510,1,1,1,29,Failed
28,الحمد الله علي نعمه العقل بعض [[الحريم]] تافهين [[وحثالات]] لابس كمامه عشان مايبي يشم [[رحتهم]] [[المروحات]] 😷,الحمد الله علي نعمه العقل بعض [[الوافدين]] تافهين [[ومو]] لابس كمامه عشان مايبي يشم [[الروائح]] [[بالبيت]] 😷,0.000989,0.008568,1,1,1,74,Failed


#### bert-attack method:

In [60]:
"""
model_name = "aubmindlab/bert-base-arabertv02-twitter",
method = "bert-attack"
threshold = 0.5
output_file = "bert-attack_results.csv"
"""
start_attack(model_name, 'bert-attack', "bert-attack_results.csv")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/Colab Notebooks/IRI/output/bert-attack_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  gradient
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM_Modified(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  20
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  False
        (compare_against_original):  True
      )
    (1): BERT(
        (metric):  cosine
        (threshold):  0.5
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
    (4): MaxModificationRate(
        (max_rate):  0.1
        (min_threshold):  4
      )
  (is_black_box):  False
) 




  0%|          | 0/710 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "

  0%|          | 1/710 [00:03<40:51,  3.46s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/710 [00:03<41:02,  3.47s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 2/710 [00:08<49:35,  4.20s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   0%|          | 2/710 [00:08<49:46,  4.22s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   0%|          | 3/710 [00:09<38:00,  3.23s/it]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:   0%|         


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 502    |
| Number of failed attacks:     | 199    |
| Number of skipped attacks:    | 9      |
| Original accuracy:            | 98.73% |
| Accuracy under attack:        | 28.03% |
| Attack success rate:          | 71.61% |
| Average perturbed word %:     | 23.11% |
| Average num. words per input: | 14.95  |
| Avg num queries:              | 43.66  |
+-------------------------------+--------+


In [61]:
bert_results = pd.read_csv(SAVE_PATH+"bert-attack_results.csv")

In [62]:
bert_results["result_type"].value_counts()

Successful    502
Failed        199
Skipped         9
Name: result_type, dtype: int64

In [63]:
bert_results[bert_results['result_type'] == 'Successful'].head(10)

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,وش ذا والله لو مو شايفات رجال حلوين [[بحياتهم]] استغفر الله [[قله]] [[حياء]] [[وتخلف]] 👎🏻👎🏻👎🏻😡😡 بايع الكليجا,وش ذا والله لو مو شايفات رجال حلوين [[حياتي]] استغفر الله [[قلى]] [[شعور]] [[وخوف]] 👎🏻👎🏻👎🏻😡😡 بايع الكليجا,0.001713,0.985722,1,0,1,72,Successful
2,بايع الكليجا ايه بعدين تعالوا قولوا انتم [[شهوانين]] 😡,بايع الكليجا ايه بعدين تعالوا قولوا انتم [[شهوانيعني]] 😡,0.016940,0.970693,1,0,1,20,Successful
6,ويعه 😤👊🏻👊🏻👊🏻 ياربي شقد [[اكررهك]] 😖😖,ويعه 😤👊🏻👊🏻👊🏻 ياربي شقد [[اكررهكك]] 😖😖,0.358952,0.848826,1,0,1,20,Successful
8,[[موت]] موت وع [[يتميلح]] بشكل [[مقرفف]] 🤢,[[الموت]] موت وع [[يتميلوح]] بشكل [[مقربب]] 🤢,0.003080,0.523456,1,0,1,58,Successful
9,عندما تغرد [[الحمير]] يكذب الكذبه و يصدقها 🐕🐕,عندما تغرد [[العصافير]] يكذب الكذبه و يصدقها 🐕🐕,0.003553,0.924632,1,0,1,17,Successful
13,سالو واحد ما هو الكاءن الذي يزن كيلو في الصباح و غرام في الليل فاجاب علي السءال المراه زوجها يقول لهالصبح قومي يا [[بقره]] 🐄 وفي الليل تعالي يا كتكوته 😂😂💔,سالو واحد ما هو الكاءن الذي يزن كيلو في الصباح و غرام في الليل فاجاب علي السءال المراه زوجها يقول لهالصبح قومي يا [[بقري]] 🐄 وفي الليل تعالي يا كتكوته 😂😂💔,0.196582,0.671780,1,0,1,20,Successful
14,عاوزين يقللوا شعبيته عشان الانتخابات الرءاسيه يقوموا يقولوا هيفرج عن مرسي وجماعته [[كفايه]] [[هبل]] [[وعبط]] [[زهقنا]] 😡,عاوزين يقللوا شعبيته عشان الانتخابات الرءاسيه يقوموا يقولوا هيفرج عن مرسي وجماعته [[كفاوكفايه]] [[هبت]] [[وعبر]] [[زها]] 😡,0.005379,0.641658,1,0,1,77,Successful
15,[[قله]] تربيه 🖕🏽,[[هه]] تربيه 🖕🏽,0.002829,0.943095,1,0,1,16,Successful
16,وهو عايش فين بروح امه ما هو بيستخدم الشارع اللي عملوه [[الكفره]] و ساكن في البيت اللي بناه الكهره ولابس الشوز اللي عمله الكفرهو ماسك الشنطهولا ده شكل للفت الانظار 😠😠,وهو عايش فين بروح امه ما هو بيستخدم الشارع اللي عملوه [[الكفرا]] و ساكن في البيت اللي بناه الكهره ولابس الشوز اللي عمله الكفرهو ماسك الشنطهولا ده شكل للفت الانظار 😠😠,0.045324,0.851373,1,0,1,19,Successful
17,ما [[اغثك]] ياخي وانت مسوي مع [[المراه]] في كل موضوع 😷,ما [[اغثم]] ياخي وانت مسوي مع [[الرجل]] في كل موضوع 😷,0.006719,0.678856,1,0,1,38,Successful


In [64]:
bert_results[bert_results['result_type'] == 'Failed'].head(10)

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
1,احلي شعور لما [[اوسع]] [[خرقك]] [[الضيق]] بزبي الكبير ياخنيث 😈👠👠 [[عمه]] كسي ديوث دياثه ممحونه كلبه كلب عمتك ساديه سالب,احلي شعور لما [[وسع]] [[خرقتك]] [[العريان]] بزبي الكبير ياخنيث 😈👠👠 [[[رابط]]] كسي ديوث دياثه ممحونه كلبه كلب عمتك ساديه سالب,0.001970,0.002547,1,1,1,64,Failed
3,والله عيب اللي صار بغض النظر جميل او شين [[وين]] كرامتك وحياءك يعني بعض البنات اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل [[تقرفت]] من بعض [[الهمج]] اللي [[بالفيديو]] بايع الكليجا الجنادريه,والله عيب اللي صار بغض النظر جميل او شين [[طيب]] كرامتك وحياءك يعني بعض البنات اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل [[قرشت]] من بعض [[الهمز]] اللي [[فيديو]] بايع الكليجا الجنادريه,0.002574,0.496475,1,1,1,77,Failed
4,هءلاء هم مايسمي علماء الوهابيه داءمانظرتهم للمراه دونيه [[وبهيمه]] كيف ترتقي الامه العربيه بهءلاء الشرذمه اللذين يسمون انفسهم ب علماء امحق علم هءلاء سفهاء ومن يصدقهم يكون [[بهيمه]] ✋🏻 اقترب [[الظهور]] العلني ل المهدي وسيحررنامن هذه [[العقول]] المعفنه 😷,هءلاء هم مايسمي علماء الوهابيه داءمانظرتهم للمراه دونيه [[وبهمه]] كيف ترتقي الامه العربيه بهءلاء الشرذمه اللذين يسمون انفسهم ب علماء امحق علم هءلاء سفهاء ومن يصدقهم يكون [[بهيلايم]] ✋🏻 اقترب [[بالظهور]] العلني ل المهدي وسيحررنامن هذه [[المعتقدات]] المعفنه 😷,0.001202,0.002567,1,1,1,77,Failed
5,[[ازق]] من كذا [[تحريض]] مافيه الله [[يمرض]] [[اشكالك]] 😷,[[ازم]] من كذا [[ضحك]] مافيه الله [[يمرمرض]] [[اشكالكك]] 😷,0.001451,0.005094,1,1,1,64,Failed
7,بايع الكليجا وصلنا لزمن نقول الله [[لايبلانا]] اعرف الحرمه من زود الحياء تستحي تحط عينها بعين اي رجال مو محرم لها [[وهذولا]] تصوير وقله حياء وتربيه [[فضحونا]] [[مكبوتات]] 😷,بايع الكليجا وصلنا لزمن نقول الله [[يستر]] اعرف الحرمه من زود الحياء تستحي تحط عينها بعين اي رجال مو محرم لها [[وبعدين]] تصوير وقله حياء وتربيه [[فضحا]] [[مكبرات]] 😷,0.001376,0.013003,1,1,1,74,Failed
10,والله ابركها من ساعه من [[لمن]] تفكريه تخليه يالمغثه كيفه [[خويه]] يابزر مالك صلاح الله ياخذ ذا [[الاشكال]] [[قررف]] متء تموتون 😷😷,والله ابركها من ساعه من [[النوم]] تفكريه تخليه يالمغثه كيفه [[خويتي]] يابزر مالك صلاح الله ياخذ ذا [[الافكار]] [[قررب]] متء تموتون 😷😷,0.001202,0.001963,1,1,1,75,Failed
11,[[ساقطه]] [[عيب]] عليك [[تقذفها]] [[يا]] كليب 🐕,[[ساقبه]] [[يعني]] عليك [[تقذفو]] [[ما]] كليب 🐕,0.001467,0.286010,1,1,1,84,Failed
12,بايع الكليجا ايوالله كلكم [[تفله]] عليكم ياللي حاقين ع [[ام]] غمزه وع [[ايفانكا]] وبياع الكليجا كلكم [[زبايل]] 😷,بايع الكليجا ايوالله كلكم [[تفكه]] عليكم ياللي حاقين ع [[امي]] غمزه وع [[ايفانزا]] وبياع الكليجا كلكم [[زباير]] 😷,0.001003,0.001525,1,1,1,77,Failed
18,انتم نجستم اليمن اي سيده تتكلم عنها يا [[معتوه]] [[زيد]] هل بنظرك علي اساس سيدك يطلع ملك قبحت انت وسيدك يا [[كلب]] اليمن العنه عليك 🖕🖕 اجلس [[فوقها]],انتم نجستم اليمن اي سيده تتكلم عنها يا [[معتوق]] [[الحوثي]] هل بنظرك علي اساس سيدك يطلع ملك قبحت انت وسيدك يا [[عار]] اليمن العنه عليك 🖕🖕 اجلس [[انت]],0.001175,0.001484,1,1,1,68,Failed
20,برنامج يسعد صباحك عمره ماسستفزني متصصل مثل سلطان اش هالاسسلوب المستفز والمنحطط [[يالطيف]] الله يعين العالم الي معاك تتعامل مو بس [[مرتك]] هذي لها الجنه بحول الله [[اعوذبالله]] من [[هالاشكال]] 😷,برنامج يسعد صباحك عمره ماسستفزني متصصل مثل سلطان اش هالاسسلوب المستفز والمنحطط [[يالطيفة]] الله يعين العالم الي معاك تتعامل مو بس [[ازواجهم]] هذي لها الجنه بحول الله [[يااختي]] من [[الالفاظ]] 😷,0.001489,0.005657,1,1,1,76,Failed
